In [11]:
from openai import OpenAI
import json

client = OpenAI()


MODEL = "gpt-4o-2024-05-13-FC"


tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
          },
          "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      },
    }
  }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=messages,
  tools=tools,
  tool_choice="auto"
)

print(completion)


ChatCompletion(id='chatcmpl-9accrUmukI9PCgxxaAAfUMacB2mnc', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_qjrXNmeCt59FCqjZmYQFnG1u', function=Function(arguments='{"location":"Boston, MA"}', name='get_current_weather'), type='function')]))], created=1718515313, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_319be4768e', usage=CompletionUsage(completion_tokens=17, prompt_tokens=80, total_tokens=97))


In [7]:
completion.json()

'{"id":"chatcmpl-9acZDK3zIjjBshuGuCmou9bZdCJSP","choices":[{"finish_reason":"tool_calls","index":0,"logprobs":null,"message":{"content":null,"role":"assistant","function_call":null,"tool_calls":[{"id":"call_SoRAXCuQVh9BQoHWk1BCPQdT","function":{"arguments":"{\\"location\\":\\"Boston, MA\\"}","name":"get_current_weather"},"type":"function"}]}}],"created":1718515087,"model":"gpt-4o-2024-05-13","object":"chat.completion","system_fingerprint":"fp_f4e629d0a5","usage":{"completion_tokens":17,"prompt_tokens":80,"total_tokens":97}}'

In [17]:
completion.choices[0].finish_reason

'tool_calls'

In [23]:
completion.choices[0].message.model_dump_json()

'{"content":null,"role":"assistant","function_call":null,"tool_calls":[{"id":"call_qjrXNmeCt59FCqjZmYQFnG1u","function":{"arguments":"{\\"location\\":\\"Boston, MA\\"}","name":"get_current_weather"},"type":"function"}]}'

### 测试non -fc直接传入返回什么

In [25]:
SYSTEM_PROMPT_FOR_CHAT_MODEL = """
    You are an expert in composing functions. You are given a question and a set of possible functions. 
    Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
    If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
    also point it out. You should only return the function call in tools call sections.
    """

USER_PROMPT_FOR_CHAT_MODEL = """
    Questions:{user_prompt}\nHere is a list of functions in JSON format that you can invoke:\n{functions}. 
    Should you decide to return the function call(s),Put it in the format of [func1(params_name=params_value, params_name2=params_value2...), func2(params)]\n
    NO other text MUST be included. """

In [29]:
data = {"question": "Calculate the area of a circle with a radius of 5 units.", "function": {"name": "geometry.calculate_area_circle", "description": "Calculate the area of a circle given its radius.", "parameters": {"type": "dict", "properties": {"radius": {"type": "integer", "description": "The radius of the circle."}, "unit": {"type": "string", "description": "The measurement unit of the radius (optional parameter, default is 'units')."}}, "required": ["radius"]}}}

# extract the question and function from the data
question = data["question"]
function = data["function"]

question

'Calculate the area of a circle with a radius of 5 units.'

In [32]:
function

{'name': 'geometry.calculate_area_circle',
 'description': 'Calculate the area of a circle given its radius.',
 'parameters': {'type': 'dict',
  'properties': {'radius': {'type': 'integer',
    'description': 'The radius of the circle.'},
   'unit': {'type': 'string',
    'description': "The measurement unit of the radius (optional parameter, default is 'units')."}},
  'required': ['radius']}}

In [33]:
revised_function = [function]
revised_function

[{'name': 'geometry.calculate_area_circle',
  'description': 'Calculate the area of a circle given its radius.',
  'parameters': {'type': 'dict',
   'properties': {'radius': {'type': 'integer',
     'description': 'The radius of the circle.'},
    'unit': {'type': 'string',
     'description': "The measurement unit of the radius (optional parameter, default is 'units')."}},
   'required': ['radius']}}]

In [34]:
type(function)

dict

In [36]:
def language_specific_pre_processing(function, test_category, string_param):
    if type(function) is dict:
        function = [function]
    if len(function) == 0:
        return function
    for item in function:
        # 一旦遇到第一个item, 就会return, 所以只会处理第一个item, 本质上是funciton掏了个list，所以就是只有一个东西， 如果是java 会修改后返回，不然直接返回，还是老样子的test file的functionkey的value
        properties = item["parameters"]["properties"]
        if test_category == "java":
            for key, value in properties.items():
                if value["type"] == "Any" or value["type"] == "any":
                    properties[key][
                        "description"
                    ] += "This parameter can be of any type of Java object."
                    properties[key]["description"] += (
                        "This is Java" + value["type"] + " in string representation."
                    )
        elif test_category == "javascript":
            for key, value in properties.items():
                if value["type"] == "Any" or value["type"] == "any":
                    properties[key][
                        "description"
                    ] += "This parameter can be of any type of Javascript object."
                else:
                    if "description" not in properties[key]:
                        properties[key]["description"] = ""
                    properties[key]["description"] += (
                        "This is Javascript "
                        + value["type"]
                        + " in string representation."
                    )
        return function  # 单纯返回一个List(Dict)
    

processed_tools = language_specific_pre_processing(revised_function, "simple", False)   
type (processed_tools)

list

In [72]:
processed_tools

[{'name': 'geometry.calculate_area_circle',
  'description': 'Calculate the area of a circle given its radius.',
  'parameters': {'type': 'dict',
   'properties': {'radius': {'type': 'integer',
     'description': 'The radius of the circle.'},
    'unit': {'type': 'string',
     'description': "The measurement unit of the radius (optional parameter, default is 'units')."}},
   'required': ['radius']}}]

In [58]:
prompt = question
# 对于java等语言，需要在prompt中加入this is java ....

# function 初始传入是个list，每个元素是一个dict，经过language_specific_pre_processing后，变成一个dict
# 原始： [{"name": "geometry.circumference", "description": "Calculate the circumference of a circle given the radius.", "parameters": {"type": "dict", "properties": {"radius": {"type": "integer", "description": "The radius of the circle."}, "units": {"type": "string", "description": "Units for the output circumference measurement. Default is 'cm'."}}, "required": ["radius"]}]
functions = processed_tools # 对于java等语言，把上面的dict的properties改一下，不重要, 反正只有一个dict包含所有东西
message = [
{
    "role": "system",
    "content": SYSTEM_PROMPT_FOR_CHAT_MODEL,
},
{
    "role": "user",
    "content": "Questions:"
    + USER_PROMPT_FOR_CHAT_MODEL.format(
        user_prompt=prompt, functions=str(functions)
    ), # 这里的functions是一个dict，变成str
},
]

response = client.chat.completions.create(
messages=message,
model="gpt-4o",)


   

In [48]:
message

[{'role': 'system',
  'content': '\n    You are an expert in composing functions. You are given a question and a set of possible functions. \n    Based on the question, you will need to make one or more function/tool calls to achieve the purpose. \n    If none of the function can be used, point it out. If the given question lacks the parameters required by the function,\n    also point it out. You should only return the function call in tools call sections.\n    '},
 {'role': 'user',
  'content': 'Questions:\n    Questions:Calculate the area of a circle with a radius of 5 units.\nHere is a list of functions in JSON format that you can invoke:\n[{\'name\': \'geometry.calculate_area_circle\', \'description\': \'Calculate the area of a circle given its radius.\', \'parameters\': {\'type\': \'dict\', \'properties\': {\'radius\': {\'type\': \'integer\', \'description\': \'The radius of the circle.\'}, \'unit\': {\'type\': \'string\', \'description\': "The measurement unit of the radius (opt

In [44]:
response.model_dump_json()

'{"id":"chatcmpl-9ad17ot7bwPP7WKFEd1CEOQ1hBOBw","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"[geometry.calculate_area_circle(radius=5)]","role":"assistant","function_call":null,"tool_calls":null}}],"created":1718516817,"model":"gpt-4o-2024-05-13","object":"chat.completion","system_fingerprint":"fp_f4e629d0a5","usage":{"completion_tokens":9,"prompt_tokens":275,"total_tokens":284}}'

In [45]:
result = response.choices[0].message.content
result

'[geometry.calculate_area_circle(radius=5)]'

### test same thing for ernie-3.5 (Not request post for EBT)

In [41]:
import erniebot
erniebot.api_type = "qianfan"
erniebot.ak = "ZL94309EFgeEgJVhc9P7XlaV"
erniebot.sk = "iFeQoBjv038BJMkIt3V4WQIyzFrOzcLT"

In [118]:
# data = {"question": "Calculate the area of a circle with a radius of 5 units.", "function": {"name": "geometry.calculate_area_circle", "description": "Calculate the area of a circle given its radius.", "parameters": {"type": "dict", "properties": {"radius": {"type": "integer", "description": "The radius of the circle."}, "unit": {"type": "string", "description": "The measurement unit of the radius (optional parameter, default is 'units')."}}, "required": ["radius"]}}}
data = {"question": "What is the capital of Brazil?", "function": [{"name": "country_info.largest_city", "description": "Fetch the largest city of a specified country.", "parameters": {"type": "dict", "properties": {"country": {"type": "string", "description": "Name of the country."}}, "required": ["country"]}}, {"name": "country_info.capital", "description": "Fetch the capital city of a specified country.", "parameters": {"type": "dict", "properties": {"country": {"type": "string", "description": "Name of the country."}}, "required": ["country"]}}, {"name": "country_info.population", "description": "Fetch the current population of a specified country.", "parameters": {"type": "dict", "properties": {"country": {"type": "string", "description": "Name of the country."}}, "required": ["country"]}}]}

# extract the question and function from the data
question = data["question"]
function = data["function"] # a dict
list_function = [function] # a list of dict
# processed_tools = language_specific_pre_processing(list_function, "multiple_functions", False) # a list of dict

ernie_message_only_tool = [

{
    "role": "user",
    "content": "Questions:"
    + USER_PROMPT_FOR_CHAT_MODEL.format(
        user_prompt=question, functions=str(function)
    )
}
]

ernie_message_list_tool = [

{
    "role": "user",
    "content": "Questions:"
    + USER_PROMPT_FOR_CHAT_MODEL.format(
        user_prompt=question, functions=str(list_function)
    )
}
]
# 这里的functions是一个dict，变成str
SYSTEM_PROMPT_FOR_CHAT_MODEL = """
    You are an expert in composing functions. You are given a question and a set of possible functions. 
    Based on the question, you will need to make one or more function/tool calls to achieve the purpose. 
    If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
    also point it out. You should only return the function call in tools call sections.
    """

In [119]:
ernie_message_only_tool

[{'role': 'user',
  'content': "Questions:\n    Questions:What is the capital of Brazil?\nHere is a list of functions in JSON format that you can invoke:\n[{'name': 'country_info.largest_city', 'description': 'Fetch the largest city of a specified country.', 'parameters': {'type': 'dict', 'properties': {'country': {'type': 'string', 'description': 'Name of the country.'}}, 'required': ['country']}}, {'name': 'country_info.capital', 'description': 'Fetch the capital city of a specified country.', 'parameters': {'type': 'dict', 'properties': {'country': {'type': 'string', 'description': 'Name of the country.'}}, 'required': ['country']}}, {'name': 'country_info.population', 'description': 'Fetch the current population of a specified country.', 'parameters': {'type': 'dict', 'properties': {'country': {'type': 'string', 'description': 'Name of the country.'}}, 'required': ['country']}}]. \n    Should you decide to return the function call(s),Put it in the format of [func1(params_name=param

In [120]:
ernie_message_list_tool

[{'role': 'user',
  'content': "Questions:\n    Questions:What is the capital of Brazil?\nHere is a list of functions in JSON format that you can invoke:\n[[{'name': 'country_info.largest_city', 'description': 'Fetch the largest city of a specified country.', 'parameters': {'type': 'dict', 'properties': {'country': {'type': 'string', 'description': 'Name of the country.'}}, 'required': ['country']}}, {'name': 'country_info.capital', 'description': 'Fetch the capital city of a specified country.', 'parameters': {'type': 'dict', 'properties': {'country': {'type': 'string', 'description': 'Name of the country.'}}, 'required': ['country']}}, {'name': 'country_info.population', 'description': 'Fetch the current population of a specified country.', 'parameters': {'type': 'dict', 'properties': {'country': {'type': 'string', 'description': 'Name of the country.'}}, 'required': ['country']}}]]. \n    Should you decide to return the function call(s),Put it in the format of [func1(params_name=par

In [129]:
ernie_response_only_tool = erniebot.ChatCompletion.create(
                messages=ernie_message_only_tool,
                system= SYSTEM_PROMPT_FOR_CHAT_MODEL,
                model="ernie-4.0",

            )

ernie_response_list_tool = erniebot.ChatCompletion.create(
                messages=ernie_message_list_tool,
                system= SYSTEM_PROMPT_FOR_CHAT_MODEL,
                model="ernie-4.0",

            )

In [130]:
ernie_response_only_tool

ChatCompletionResponse(rcode=200, rbody={'id': 'as-aqipasideb', 'object': 'chat.completion', 'created': 1718525108, 'result': '[country_info.capital(country="Brazil")]', 'is_truncated': False, 'need_clear_history': False, 'finish_reason': 'normal', 'usage': {'prompt_tokens': 370, 'completion_tokens': 11, 'total_tokens': 381}}, rheaders={'Access-Control-Allow-Headers': 'Content-Type', 'Access-Control-Allow-Origin': '*', 'Appid': '78879075', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json; charset=utf-8', 'Date': 'Sun, 16 Jun 2024 08:05:08 GMT', 'P3p': 'CP=" OTI DSP COR IVA OUR IND COM "', 'Server': 'Apache', 'Set-Cookie': 'BAIDUID=6C19B86631A1A6C4BC8D7C93CB278708:FG=1; expires=Thu, 31-Dec-37 23:55:55 GMT; max-age=2145916555; path=/; domain=.baidu.com; version=1', 'Statement': 'AI-generated', 'Vary': 'Accept-Encoding', 'X-Aipe-Self-Def': 'eb_total_tokens:381,prompt_tokens:370,id:as-aqipasideb', 'X-Baidu-Request-Id': '807be993d70c87c65c4e3c9999d96

In [131]:
getted_result = ernie_response_only_tool.get_result()

type(getted_result)

str

In [124]:
getted_result

'[country_info.capital(country="Brazil")]'

In [125]:
ernie_response_list_tool.get_result()

'```json\n[\n  {\n    "tool_call": "country_info.capital(country=\'Brazil\')"\n  }\n]\n```'

In [126]:
print(ernie_response_list_tool.get_result())

```json
[
  {
    "tool_call": "country_info.capital(country='Brazil')"
  }
]
```


In [128]:
print(type(ernie_response_only_tool.result), type(ernie_response_list_tool.result))
print(ernie_response_only_tool.result)
print(ernie_response_list_tool.result)

<class 'str'> <class 'str'>
[country_info.capital(country="Brazil")]
```json
[
  {
    "tool_call": "country_info.capital(country='Brazil')"
  }
]
```


In [114]:
### 清理ernie resposne

# 假设这是你的原始输出
original_output = getted_result = ernie_response_only_tool.get_result()
print(f"original_output \n: {original_output}")
# 去除开头的'```\n' 和结尾的'\n```'
cleaned_output = original_output.replace('```\n', '').replace('\n```', '')

# 打印清理后的输出
print(f"cleaned_output: {cleaned_output}")
print(cleaned_output)

original_output 
: ```
[geometry.calculate_area_circle(radius=5)]
```
cleaned_output: [geometry.calculate_area_circle(radius=5)]
[geometry.calculate_area_circle(radius=5)]


In [95]:
type(response.choices[0].message.content)
print(response.choices[0].message.content)


[geometry.calculate_area_circle(radius=5)]


In [132]:
import erniebot

# List supported models
models = erniebot.Model.list()

In [133]:
models

[('ernie-3.5', '文心大模型（ernie-3.5）'),
 ('ernie-turbo', '文心大模型（ernie-turbo）'),
 ('ernie-4.0', '文心大模型（ernie-4.0）'),
 ('ernie-3.5-8k', '文心大模型（ernie-3.5-8k）'),
 ('ernie-speed', ' 文心大模型（ernie-speed）'),
 ('ernie-speed-128k', ' 文心大模型（ernie-speed-128k）'),
 ('ernie-tiny-8k', ' 文心大模型（ernie-tiny-8k）'),
 ('ernie-char-8k', ' 文心大模型（ernie-char-8k）'),
 ('ernie-text-embedding', '文心百中语义模型'),
 ('ernie-vilg-v2', '文心一格模型')]

In [6]:
import json
import argparse, json, os
from tqdm import tqdm
from model_handler.handler_map import handler_map
from model_handler.model_style import ModelStyle
from model_handler.constant import USE_COHERE_OPTIMIZATION

def convert_yewu_jsonl_to_BFCL_jsonl(originial_yewu_file, coverted_simple_ast_test, coverted_multiple_ast_test):
    with open(originial_yewu_file, 'r') as originial_file, \
         open(coverted_simple_ast_test, 'w') as simple_file, \
         open(coverted_multiple_ast_test, 'w') as multiple_file:
        
        data = originial_file.readlines()

        for line in data:
            line = json.loads(line) 
            if len(line['tools']) > 1: # multiple
                this_line = {"question": "", "function": []}
                for tool in line["tools"]:
                    this_line["function"].append(tool["function"])
                multiple_file.write(json.dumps(this_line) + "\n")
            else: # simple
                this_line = {"question": "", "function": line["tools"][0]["function"]}
                simple_file.write(json.dumps(this_line) + "\n")

# Example usage:
originial_yewu_file = '/Users/gaoyukun01/E/job4_folder/gorilla_justin/berkeley-function-call-leaderboard/yewu.jsonl'
coverted_simple_ast_test = '/Users/gaoyukun01/E/job4_folder/gorilla_justin/berkeley-function-call-leaderboard/coverted_simple_ast_test.jsonl'
coverted_multiple_ast_test = '/Users/gaoyukun01/E/job4_folder/gorilla_justin/berkeley-function-call-leaderboard/coverted_multiple_ast_test.jsonl'


convert_yewu_jsonl_to_BFCL_jsonl(originial_yewu_file, coverted_simple_ast_test, coverted_multiple_ast_test)
